# Supervised Learning Classification Model - Testing if change in one day average price and seven day average can predict whether NFT price increases

## Imports

Making the necessary imports

In [14]:
import pandas as pd 
import os 
from dotenv import load_dotenv
import requests 
import json 
from pathlib import Path
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from imblearn.metrics import classification_report_imbalanced
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import hvplot.pandas


## Importing the API

See if the .env file is working correctly

In [15]:
load_dotenv()

True

In [16]:
api_key = os.getenv('BLOCKSPAN_API_KEY')

Retrieve the API and gather the data in a data frame.

In [17]:
url = "https://api.blockspan.com/v1/exchanges/collectionsranking?chain=eth-main&exchange=opensea&ranking=seven_day_volume&page_size=100"
headers = {
    "accept": "application/json",
    "X-API-KEY": api_key
}

In [18]:
response = requests.get(url, headers=headers).json()
#print(json.dumps(response, indent=4, sort_keys=True))

In [19]:
nft_df = pd.DataFrame(response['results'])
nft_df.head()

,key,name,description,update_at,total_volume,one_day_volume,seven_day_volume,thirty_day_volume,total_sales,one_day_sales,seven_day_sales,thirty_day_sales,total_average_price,one_day_average_price,seven_day_average_price,thirty_day_average_price,contracts
0,valhalla,Valhalla,Valhalla is a crypto native brand for gamers. ...,2022-12-19T02:58:00.928Z,11262.738558373669,448.75581168969853,3967.3500461705735,11262.738558373685,12294,356,2827,12294,0.9161166876829078,1.2605500328362318,1.4033781557023606,0.9161166876829091,[{'contract_address': '0x231d3559aa848bf10366f...
1,boredapeyachtclub,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,2022-12-19T01:55:59.554Z,699316.0191672856,103,3074.5875698799896,11176.702116379995,27544,2,44,150,25.389050942756523,51.5,69.87699022454522,74.51134744253329,[{'contract_address': '0xbc4ca0eda7647a8ab7c20...
2,art-blocks,Art Blocks,Art Blocks is dedicated to bringing compelling...,2022-12-15T19:49:23.602Z,463304.69361787103,281.12792500000006,2136.789362268989,9127.35848823819,207755,200,1662,6494,2.230053156929417,1.4056396250000003,1.2856735031702702,1.4055063887031396,[{'contract_address': '0xa7d8d9ef8d8ce8992df33...
3,cryptopunks,CryptoPunks,"CryptoPunks launched as a fixed set of 10,000 ...",2022-12-19T01:12:50.624Z,1075764.6970093388,140.95,1979.2100000000003,12012.210000000003,22060,2,29,176,48.76539877648861,70.475,68.24862068965518,68.2511931818182,[{'contract_address': '0xb47e3cd837ddf8e4c57f0...
4,mutant-hound-collars,Mutant Hound Collars,"Mutant Hounds are a collection of 10,000 lore-...",2022-12-19T02:30:13.213Z,1557.1282909447996,469.4254873900003,1557.1282909447925,1557.1282909447925,3855,818,3855,3855,0.40392432968736697,0.5738697889853305,0.40392432968736514,0.40392432968736514,[{'contract_address': '0xae99a698156ee8f8d07cb...


## Setting the target for ML

Calculating if the NFT price increases or decreases

In [20]:
nft_df['change'] = nft_df['one_day_average_price'].astype(float) - nft_df['seven_day_average_price'].astype(float)
nft_df.head()
#If positive, then stock increased
#If negative, stock fell

,key,name,description,update_at,total_volume,one_day_volume,seven_day_volume,thirty_day_volume,total_sales,one_day_sales,seven_day_sales,thirty_day_sales,total_average_price,one_day_average_price,seven_day_average_price,thirty_day_average_price,contracts,change
0,valhalla,Valhalla,Valhalla is a crypto native brand for gamers. ...,2022-12-19T02:58:00.928Z,11262.738558373669,448.75581168969853,3967.3500461705735,11262.738558373685,12294,356,2827,12294,0.9161166876829078,1.2605500328362318,1.4033781557023606,0.9161166876829091,[{'contract_address': '0x231d3559aa848bf10366f...,-0.142828
1,boredapeyachtclub,Bored Ape Yacht Club,The Bored Ape Yacht Club is a collection of 10...,2022-12-19T01:55:59.554Z,699316.0191672856,103,3074.5875698799896,11176.702116379995,27544,2,44,150,25.389050942756523,51.5,69.87699022454522,74.51134744253329,[{'contract_address': '0xbc4ca0eda7647a8ab7c20...,-18.376990
2,art-blocks,Art Blocks,Art Blocks is dedicated to bringing compelling...,2022-12-15T19:49:23.602Z,463304.69361787103,281.12792500000006,2136.789362268989,9127.35848823819,207755,200,1662,6494,2.230053156929417,1.4056396250000003,1.2856735031702702,1.4055063887031396,[{'contract_address': '0xa7d8d9ef8d8ce8992df33...,0.119966
3,cryptopunks,CryptoPunks,"CryptoPunks launched as a fixed set of 10,000 ...",2022-12-19T01:12:50.624Z,1075764.6970093388,140.95,1979.2100000000003,12012.210000000003,22060,2,29,176,48.76539877648861,70.475,68.24862068965518,68.2511931818182,[{'contract_address': '0xb47e3cd837ddf8e4c57f0...,2.226379
4,mutant-hound-collars,Mutant Hound Collars,"Mutant Hounds are a collection of 10,000 lore-...",2022-12-19T02:30:13.213Z,1557.1282909447996,469.4254873900003,1557.1282909447925,1557.1282909447925,3855,818,3855,3855,0.40392432968736697,0.5738697889853305,0.40392432968736514,0.40392432968736514,[{'contract_address': '0xae99a698156ee8f8d07cb...,0.169945


In [21]:
nft_df = nft_df

Cleaning up the dataframe. Dropping unesessary columns and assigning NFTs that increased in price a 1 and NFTS that fell in price a 0 in the 'target' column. 

In [22]:
nft_df['target'] = nft_df['change'].apply(lambda x: 0 if x <= 0 else 1)

In [23]:
new_nft_df = nft_df.set_index('key')
new_nft_df = nft_df.drop(columns=['update_at','name','description','contracts','key','total_volume','one_day_volume','seven_day_volume','thirty_day_volume','total_sales','one_day_sales','one_day_sales','seven_day_sales','thirty_day_sales','total_average_price','thirty_day_average_price'])
new_nft_df.head()

,one_day_average_price,seven_day_average_price,change,target
0,1.2605500328362318,1.4033781557023606,-0.142828,0
1,51.5,69.87699022454522,-18.376990,0
2,1.4056396250000003,1.2856735031702702,0.119966,1
3,70.475,68.24862068965518,2.226379,1
4,0.5738697889853305,0.40392432968736514,0.169945,1


Visualizing the dataframe.

In [54]:
new_nft_df.hvplot.scatter( x = 'index', y = 'change', by = 'target')

:NdOverlay   [target]
   :Scatter   [index]   (change)

## Creating the supervised learning model.

In [26]:
# The y variable should focus on the target column
y = new_nft_df['target'].astype(float)
# The X variable should include all features except the target
x = new_nft_df.drop(columns=['target']).astype(float)

In [27]:
X_train, X_test, y_train, y_test = train_test_split(x, y)

In [28]:
#Scaling data sets
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
#trans X train and test sets using scaler
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

In [29]:
# Declare a logistic regression model.
# Apply a random_state of 9 to the model
logistic_regression_model = LogisticRegression(random_state=9)
logistic_regression_model

LogisticRegression(random_state=9)

In [30]:
# Fit and save the logistic regression model using the training data
lr_model = logistic_regression_model.fit(X_train, y_train)

In [31]:
#Generate training predictions
training_predictions = lr_model.predict(X_train)

#Generate testing predictions
testing_predictions = logistic_regression_model.predict(X_test)

## Creating the Confusion Matrix

In [32]:
# Create and save the confustion matrix for the training data
training_matrix = confusion_matrix(y_train, training_predictions)

# Print the confusion matrix for the training data
print(training_matrix)

[[47  1]
 [24  3]]


In [33]:
# Create and save the confustion matrix for the testing data
test_matrix = confusion_matrix(y_test, testing_predictions)

# Print the confusion matrix for the testing data
print(test_matrix)

[[15  1]
 [ 8  1]]


In [34]:
# Create and save the training classifiction report
training_report = classification_report(y_train, training_predictions)

# Print the training classification report
print(training_report)

              precision    recall  f1-score   support

         0.0       0.66      0.98      0.79        48
         1.0       0.75      0.11      0.19        27

    accuracy                           0.67        75
   macro avg       0.71      0.55      0.49        75
weighted avg       0.69      0.67      0.58        75



In [35]:
# Create and save the testing classifiction report
testing_report = classification_report(y_test, testing_predictions)

# Print the testing classification report
print(testing_report)

              precision    recall  f1-score   support

         0.0       0.65      0.94      0.77        16
         1.0       0.50      0.11      0.18         9

    accuracy                           0.64        25
   macro avg       0.58      0.52      0.48        25
weighted avg       0.60      0.64      0.56        25



## Comparing predictions to the real data

In [36]:
target_only = new_nft_df['target']

# Convert those predictions (and actual values) to a DataFrame
results_df = pd.DataFrame({"prediction": testing_predictions.astype(int)})

In [37]:
outcome_df = pd.concat([new_nft_df, results_df], axis=1)
outcome_df = outcome_df.dropna()

In [38]:
outcome_df

,one_day_average_price,seven_day_average_price,change,target,prediction
0,1.2605500328362318,1.4033781557023606,-0.142828,0,0.0
1,51.5,69.87699022454522,-18.376990,0,0.0
2,1.4056396250000003,1.2856735031702702,0.119966,1,1.0
3,70.475,68.24862068965518,2.226379,1,0.0
4,0.5738697889853305,0.40392432968736514,0.169945,1,0.0
5,14.936336363636366,15.235241063157902,-0.298905,0,1.0
6,3.1553327872340424,2.731201470296856,0.424131,1,0.0
7,0.4686463846153846,0.4762203532428783,-0.007574,0,0.0
8,30.083333333333332,30.853333333333335,-0.770000,0,0.0
9,0.7540261290322582,1.0375829648542294,-0.283557,0,0.0


## Conclusions

The predictions resulting from the change in one day average price and seven day average price cannot accurately predict the trajectory of NFT price in this model. However, it was a good introduction and our team would like experiemnting with different variables/metrics in the future.